In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/17 17:30:20 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 192.168.146.135 instead (on interface eth0)
22/08/17 17:30:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/17 17:30:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols, glm

In [3]:
yellow_spark = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
yellow_spark.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- is_weekend: boolean (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- trip_length: double (nullable = true)
 |-- Start(ET): string (nullable = true)
 |-- Attend.: string (nullable = true)
 |-- Win: string (nullable = true)
 |-- pickup_hourly: string (nullable = true)
 |-- dropoff_hourly: string (nullable = true)



In [12]:
yellow_spark.groupBy(F.col("Pickup_hourly")).count().show(30)

+-------------+------+
|Pickup_hourly| count|
+-------------+------+
|        14:00|338637|
|        12:00|328282|
|        20:00|376491|
|        06:00|120459|
|        21:00|366736|
|        01:00|164805|
|        03:00| 78804|
|        09:00|304959|
|        05:00| 43935|
|        11:00|311909|
|        15:00|342394|
|        13:00|324145|
|        02:00|120178|
|        19:00|417866|
|        08:00|301624|
|        22:00|350432|
|        07:00|234182|
|        18:00|436951|
|        16:00|319355|
|        23:00|267534|
|        10:00|299904|
|        00:00|222410|
|        04:00| 46034|
|        17:00|379912|
+-------------+------+



In [33]:
yellow_weekday = yellow_spark.filter( F.col("is_weekend") == True )
yellow_weekend = yellow_spark.filter( F.col("is_weekend") == False)

yellow_win = yellow_spark.filter( F.col("win") == "Yes" )
yellow_lose = yellow_spark.filter( F.col("win") == "No" )



In [14]:
from pyspark.sql.functions import *
(yellow_spark
 .groupBy(F.col("Pickup_hourly").alias("hour"))
 .agg(mean("tip_amount").alias("mean_tip"))
 .show(30)
)

+-----+------------------+
| hour|          mean_tip|
+-----+------------------+
|14:00|2.2072910520705777|
|12:00|2.2127620765074814|
|20:00|2.3382355222303373|
|06:00|1.9312098722387012|
|21:00|2.4110341226386884|
|01:00|2.3371176238585316|
|03:00| 2.294972082635396|
|09:00|2.1508834958142637|
|05:00| 2.135467395015362|
|11:00|2.2136596892043063|
|15:00|2.1974344176592653|
|13:00|2.2091713893474405|
|02:00| 2.293484165155034|
|19:00| 2.305038050475544|
|08:00|2.1058888881522004|
|22:00| 2.459709558487872|
|07:00| 2.011141206412145|
|18:00| 2.311173083480809|
|16:00|  2.27180269605932|
|23:00|2.4635157400555445|
|10:00|2.1804134989864146|
|00:00|2.4242085337890105|
|04:00| 2.372181865577613|
|17:00|2.3028187843501344|
+-----+------------------+



In [34]:
# average tip amount per hour in a weekday
(yellow_weekday
 .groupBy(hour("Pickup_hourly").alias("hour"))
 .agg(mean("tip_amount").alias("mean_tip"))
 .show()
)

+----+------------------+
|hour|          mean_tip|
+----+------------------+
|  12|2.1626819055201585|
|  22| 2.446235691321408|
|   1|2.3254755988302573|
|  13| 2.176139068634324|
|   6| 2.139035560671228|
|  16|2.2606291177283406|
|   3| 2.274831516671488|
|  20|2.3287920516273064|
|   5| 2.312550354345394|
|  19| 2.264033660544166|
|  15| 2.224087137265075|
|  17| 2.281596101551816|
|   9| 2.006334670956713|
|   4|2.3428525984354143|
|   8| 2.012010099542251|
|  23|2.4537930505652645|
|   7|2.0283621560006453|
|  10| 2.062244875098028|
|  21|2.3846844674469443|
|  11|2.1112948192322296|
+----+------------------+
only showing top 20 rows



In [35]:
# average tip amount per hour in a weekend

(yellow_weekend
 .groupBy(hour("Pickup_hourly").alias("hour"))
 .agg(mean("tip_amount").alias("mean_tip"))
 .show()
)


+----+------------------+
|hour|          mean_tip|
+----+------------------+
|  12|2.2458609367016087|
|  22| 2.465508696308898|
|   1|2.3667451764857774|
|  13|  2.23097526874565|
|   6|1.8982083076804819|
|  16| 2.278479554981979|
|   3| 2.370431875677628|
|  20| 2.342019114867856|
|   5| 2.078286411128841|
|  19|2.3225182269978872|
|  15|2.1823695085714725|
|  17| 2.313739749237615|
|   9| 2.204921297798007|
|   4|2.4315203257371736|
|   8| 2.126849155621941|
|  23|2.4685516834425147|
|   7| 2.008542537977298|
|  10|2.2460774457931643|
|  21| 2.421188850021614|
|  11| 2.278307719669036|
+----+------------------+
only showing top 20 rows



ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.